# FUNCTION glcmdesc

The glcmdesc function computes the descriptors from the gray level co-occurence matrix.


<strong> glcm, v = glcmdesc(f, offset=[],mask=[]) </strong>
<ul>
    <li> <strong> Output </strong> </li>
        <ul> 
            <li> <strong> v: </strong>array with GLCM descriptors.</li>
            <li> <strong> glcm: </strong> Gray Level Co-Ocurrence Matrix. </li>
        </ul> 
    <li> <strong> Input </strong> </li>
        <ul>
            <li> <strong> f: </strong> input image (2D or 3D) </li>
            <li> <strong> offset: </strong> defines the distance and orientation. </li>
            <li> <strong> mask: </strong> defines the mask. </li>
        </ul>
</ul>

<strong> 2D offsets </strong>

|Directions| Degrees |
|-----------|-------------|
| [0, D]   | zero degrees |
| [D, -D]  | 45 degrees   |          
| [D, 0]   | 90 degrees   | 
| [D, D]   | 135 degrees  | 

<strong> 3D offsets </strong>
Standard 2D directions

|Directions| Degrees |
|-----------|-------------|
| [0, D]   | zero degrees |
| [D, -D]  | 45 degrees   |          
| [D, 0]   | 90 degrees   | 
| [D, D]   | 135 degrees  | 

Additional 9 directions


| Directions  | Degrees zy  | Degrees yx  |
|-------------|-------------|-------------|
| [D 0 -D]    | 0 degrees   | 45 degrees  |
| [D 0 0]     | straight up |             |
| [D 0 D]     | 0 degrees   | 135 degrees |
| [D D 0]     | 90 degrees  | 45 degrees  |
| [D -D 0]    | 90 degrees  | 135 degrees |
| [D D -D]    | 45 degrees  | 45 degrees  |
| [D -D D]    | 45 degrees  | 135 degrees |
| [D D D]     | 135 degrees | 45 degrees  |
| [D -D -D]   | 135 degrees | 135 degrees |

The glcmdesc function extracts from the co-occorence matrix a set of descriptors.

# FUNCTION CODE

In [1]:
import numpy as np
import nbimporter
from auxiliary_functions import *
from glcm import *

def glcmdesc(f, offset=[], mask=[]):
        lev = int(f.max()+1)  # levels
        if mask != []: # if there is a mask, it is necessary to set a invalid value to all the pixels that are not in the mask
            aux = f.copy()
            hist,_ = np.histogram(np.ravel(f[mask]), bins=lev, range=(0,lev-1)) #look for an empty level inside the mask
            if np.argwhere(hist==0).shape[0] != 0:  #if there is an empty level
                new_v = np.argwhere(hist==0).max() # it is selected the highest empty gray level
            else:  # if there isn't an empty level
                new_v = lev # the chosen invalid is f.max()+1
                lev+=1
            aux[~mask] = new_v # all pixels that are not inside the mask receive this invalid value
            g = glcm(lev, offset, aux) # the glcm matrix is computed
            g[new_v,:] = 0 #the rows and columns that corresponds to the invalid value (outside the mask) are nulled
            g[:,new_v] = 0 #it is important to notice that null rows and columns do not change the descriptors
        else: # if there is no mask
            g = glcm(lev, offset, f)
            
        
        ## glcm normalization ##
        if g.sum() != 0:
            g = g.astype(float)/g.sum()
            
        ## compute auxiliary variables ##
        (num_level, num_level2) = g.shape
        I, J = np.ogrid[0:num_level, 0:num_level]
        I = 1+ np.array(range(num_level)).reshape((num_level, 1))
        J = 1+ np.array(range(num_level)).reshape((1, num_level))
        diff_i = I - np.apply_over_axes(np.sum, (I * g), axes=(0, 1))[0, 0]
        diff_j = J - np.apply_over_axes(np.sum, (J * g), axes=(0, 1))[0, 0]
        std_i = np.sqrt(np.apply_over_axes(np.sum, (g * (diff_i) ** 2),axes=(0, 1))[0, 0])
        std_j = np.sqrt(np.apply_over_axes(np.sum, (g * (diff_j) ** 2),axes=(0, 1))[0, 0])
        cov = np.apply_over_axes(np.sum, (g * (diff_i * diff_j)),axes=(0, 1))[0, 0]
        
        gxy = np.zeros(2*g.shape[0]-1)   ### g x+y
        gx_y = np.zeros(g.shape[0])  ### g x-y
        for i in xrange(g.shape[0]):
            for j in xrange(g.shape[0]):
                gxy[i+j] += g[i,j]
                gx_y[np.abs(i-j)] += g[i,j]
                
        mx_y = (gx_y*np.arange(len(gx_y))).sum()
        v = np.zeros(11)
        i,j = np.indices(g.shape)+1
        ii = np.arange(len(gxy))+2
        ii_ = np.arange(len(gx_y))
        
        ## compute descriptors ##
        v[0] = np.apply_over_axes(np.sum, (g ** 2), axes=(0, 1))[0, 0] # Angular second moment
        v[1] = np.apply_over_axes(np.sum, (g * ((I - J) ** 2)), axes=(0, 1))[0, 0] # Contrast
        if std_i>1e-15 and std_j>1e-15: # handle the special case of standard deviations near zero
            v[2] = cov/(std_i*std_j)#v[2] = greycoprops(g,'correlation') # Correlation
        else:
            v[2] = 1
        v[3] = np.apply_over_axes(np.sum, (g* (diff_i) ** 2),axes=(0, 1))[0, 0]# Sum of squares
        v[4] = np.sum(g * (1. / (1. + (I - J) ** 2))) # Inverse difference moment
        v[5] = (gxy*ii).sum() # Sum average
        v[6] = ((ii-v[5])*(ii-v[5])*gxy).sum() # Sum variance
        v[7] = -1*(gxy*np.log10(gxy+ np.spacing(1))).sum() # Sum entropy
        v[8] = -1*(g*np.log10(g+np.spacing(1))).sum() # Entropy
        v[9] = ((ii_-mx_y)*(ii_-mx_y)*gx_y).sum() # Difference variance
        v[10] = -1*(gx_y*np.log10(gx_y++np.spacing(1))).sum() # Difference entropy
        
        return g, v

Importing Jupyter notebook from auxiliary_functions.ipynb
Importing Jupyter notebook from glcm.ipynb
Importing Jupyter notebook from glcmdesc.ipynb


# EXAMPLES

<strong> 2D Examples </strong>


Numeric Example:

In [24]:
import nbimporter
from auxiliary_functions import *
from glcmdesc import *
f = np.array( [[2,1,1,0,0,0,0,0,0],
               [1,2,0,0,0,2,0,1,0],
               [1,0,2,1,0,2,0,1,0],
               [0,0,0,2,0,1,1,0,0]], dtype=np.uint8)
offset=[]
lev = int(f.max()+1)  # levels
g = glcm(lev, offset, f)

glcm, v = glcmdesc(f, offset)
print 'Gray Level Co-occurence Matrix descriptors'
print v
print
print

len(img)
2
len(img)
2
Gray Level Co-occurence Matrix descriptors
[ 0.18164062  1.375      -0.16984388  0.60058594  0.6125      3.1875
  0.96484375  0.56703915  0.81387275  0.609375    0.46999155]




Numeric Example with a Different Orientation:

In [22]:
f = np.array( [[2,1,1,0,0],
               [1,2,0,0,0],
               [1,0,2,1,0],
               [0,0,0,2,0]], dtype=np.uint8)

offset = np.array([1,1],dtype=np.int32)
glcm, v = glcmdesc(f,offset)
print 'Gray Level Co-occurence Matrix descriptors'
print v
print
print

len(img)
2
Gray Level Co-occurence Matrix descriptors
[ 0.30555556  0.5         0.71095787  0.57638889  0.75        3.5         2.25
  0.45154499  0.54938312  0.25        0.30103   ]




<strong> Examples with Mask </strong> 

In [20]:
f  = np.array( [[1,2,2,2],
                [2,2,1,1],
                [1,2,0,1],
                [2,1,0,1]], dtype=np.uint8)

glcm,v = glcmdesc(f,mask = f!=1)
print 'Gray Level Co-occurence Matrix descriptors'
print v
print
print

len(img)
2
Gray Level Co-occurence Matrix descriptors
[ 0.625       1.          1.          0.          0.8         5.5         0.75
  0.24421905  0.24421905  0.75        0.24421905]




Comparison between matrices and descriptors using mask or using no mask

In [6]:
f  = np.array( [[2,2,2,1],
                [2,2,0,1],
                [0,2,0,1],
                [1,1,1,1]], dtype=np.uint8)
print len(f.shape)
glcm,v = glcmdesc(f,mask = f!=1)
print 'Gray Level Co-occurence Matrix descriptors'
print v

2
len(img)
2
Gray Level Co-occurence Matrix descriptors
[ 0.38888889  2.         -0.31622777  0.55555556  0.6         5.          1.
  0.30103     0.43924729  1.          0.30103   ]


In [18]:
f  = np.array( [[2,2,2],
                [2,2,0],
                [0,2,0]], dtype=np.uint8)
print len(f.shape)
glcm,v = glcmdesc(f)
print 'Gray Level Co-occurence Matrix descriptors'
print v
print

2
len(img)
2
Gray Level Co-occurence Matrix descriptors
[ 0.38888889  2.         -0.31622777  0.55555556  0.6         5.          1.
  0.30103     0.43924729  1.          0.30103   ]



<strong> 3D Examples </strong>

In [17]:
f = np.array([[[1,2,2,0,0,1],[0,0,1,2,2,1],[1,1,0,0,0,2],[1,1,1,2,2,2],[1,1,2,2,0,0]],
             [[1,2,2,0,0,1],[0,0,1,2,2,1],[1,1,0,0,0,2],[1,1,1,2,2,2],[1,1,2,2,0,0]],
             [[1,2,2,0,0,1],[0,0,1,2,2,1],[1,1,0,0,0,2],[1,1,1,2,2,2],[1,1,2,2,0,0]]], dtype=np.uint8)
glcm,v = glcmdesc(f) # orientation 0 degrees
print 'Gray Level Co-occurence Matrix descriptors'
print v
print
print

len(img)
3
Gray Level Co-occurence Matrix descriptors
[ 0.1488      0.8         0.41432451  0.64        0.744       4.08        1.9136
  0.684676    0.87752801  0.4864      0.40986496]




Different Orientation

In [9]:
glcm,v = glcmdesc(f,offset = np.array([0,1,-1], dtype = np.int32)) # orientation 45 degrees
print 'Gray Level Co-occurence Matrix descriptors'
print v

len(img)
3
Gray Level Co-occurence Matrix descriptors
[ 0.18        1.7        -0.31083494  0.69        0.57        4.1         0.89
  0.48195333  0.78379231  0.7         0.47601599]


In [10]:
glcm,v = glcmdesc(f,offset = np.array([1,0,0], dtype = np.int32)) # straigh up
print 'Gray Level Co-occurence Matrix descriptors'
print v

len(img)
3
Gray Level Co-occurence Matrix descriptors
[  3.35555556e-01   0.00000000e+00   1.00000000e+00   6.32222222e-01
   1.00000000e+00   4.06666667e+00   2.52888889e+00   4.75671184e-01
   4.75671184e-01   0.00000000e+00  -9.64327467e-17]


In [16]:
glcm,v = glcmdesc(f,offset = np.array([3,0,-3], dtype = np.int32)) # straigh up
print 'Gray Level Co-occurence Matrix descriptors'
print v
print
print

len(img)
3
Gray Level Co-occurence Matrix descriptors
[ 0.  0.  1.  0.  0.  0.  0. -0. -0.  0. -0.]




Using Mask:

In [13]:
glcm,v = glcmdesc(f,offset = np.array([0,1,-1], dtype = np.int32),mask=f!=1) # orientation 45 degrees
print 'Gray Level Co-occurence Matrix descriptors'
print v

len(img)
3
Gray Level Co-occurence Matrix descriptors
[ 0.38        2.8        -0.5         0.64        0.44        4.6         0.84
  0.265295    0.44717262  0.84        0.265295  ]


In [14]:
glcm,v = glcmdesc(f,offset = np.array([1,0,0], dtype = np.int32),mask=f!=1) # straigh up
print 'Gray Level Co-occurence Matrix descriptors'
print v

len(img)
3
Gray Level Co-occurence Matrix descriptors
[  5.01385042e-01   0.00000000e+00   1.00000000e+00   9.97229917e-01
   1.00000000e+00   4.10526316e+00   3.98891967e+00   3.00428202e-01
   3.00428202e-01   0.00000000e+00  -9.64327467e-17]


# EQUATION

The co-occurence matrix becomes the estimate of the joint probability $ p_{d {\theta}} $ of two pixels, a distance $d$ apart along a given direction $\theta$ having particular (co-occurring) values $i$ and $j$.

Let ${\mu}_x$, ${\mu}_y$ and $\rho_x$, $\rho_y$ denote the mean and standard deviations of the row and column sums of the co-occurence matrix, respectively, related to the marginal distribuitions $p_x(i)$ and $p_y(i)$, and $N_g$ the number of discrete intensity levels.

<strong>Angular second moment </strong>
$$ AngScMom =  \sum\limits_{i=1}^{N_g} \sum\limits_{j=1}^{N_g} p(i,j)^2 $$ 

<strong> Contrast </strong>
$$ Contrast = \sum\limits_{n=0}^{N_g-1}n^2 \sum\limits_{i=1}^{N_g} \sum\limits_{j=1}^{N_g} p(i,j) $$ 
 $ where $ $n=|i-j|$ 
 
<strong> Sum of Squares </strong>
$$ SumOfSqs = \sum\limits_{i=1}^{N_g} \sum\limits_{j=1}^{N_g} (i-\mu_x)^2 p(i,j) $$ 

<strong> Inverse Difference Moment </strong>
$$ InvDfMom = \sum\limits_{i=1}^{N_g} \sum\limits_{j=1}^{N_g} \frac{1}{1+(i+j)^2} p(i,j) $$ 

<strong> Sum Average </strong>
$$ SumAverg = \sum\limits_{i=1}^{2 N_g} ip_{x+y}(i) $$ 
$where$ $p_{x+y}(k) =  \sum\limits_{i=1}^{N_g}\sum\limits_{j=1}^{N_g} p(i,j) $ $k=i+j$